In [1]:
import geopandas as gpd
import pandas as pd
import pyproj
from shapely.ops import transform
import utm

## Open GIS Data

In [2]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)
states_df = states_df.to_crs(4326)

## Get State Population data

In [3]:
state_populations = pd.read_excel(
    "data/NST-EST2024-POP.xlsx", sheet_name=None, engine="openpyxl"
)

In [4]:
state_populations_df = state_populations["NST-EST2024-POP"][
    [
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)",
        "Unnamed: 5",
    ]
]
state_populations_df = state_populations_df.rename(
    columns={
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)": "NAME",
        "Unnamed: 5": "POPULATION",
    }
)
state_populations_df["NAME"] = state_populations_df["NAME"].str[1:]

In [5]:
states_with_population_df = states_df.merge(state_populations_df, on="NAME", how="left")
states_with_population_df = states_with_population_df[["STUSPS", "POPULATION"]]

In [6]:
states_with_population_df = states_with_population_df.dropna()

## Open Interstate data

In [7]:
highways_gdf = gpd.read_file(
    "data/NTAD_National_Highway_System_-1015841555521948228.gpkg"
)

/mnt/c/Users/Winston/Documents/GitHub/VineMapper/.venv/lib/python3.11/site-packages/pyogrio/raw.py:198: RuntimeWarning: Non-conformant content for record 1 in column FUT_YEAR, 2039-12-31T00:00:00.0Z, successfully parsed
  return ogr_read(


In [32]:
interstates_gdf = highways_gdf[highways_gdf["SIGNT1"] == "I"]
interstates_gdf = interstates_gdf[["geometry"]].reset_index()

In [41]:
usa_interstates = interstates_gdf.union_all()
interstate_union = gpd.GeoDataFrame({"geometry": [usa_interstates]}, crs=4326)

In [43]:
interstate_union.to_file("data/interstate.gpkg")

In [8]:
interstates_gdf = highways_gdf[highways_gdf["SIGNT1"] == "I"]
interstates_gdf = interstates_gdf[["geometry"]].reset_index()
interstates_gdf.to_file("data/interstates.gpkg")
interstates_gdf = interstates_gdf.to_crs(4326)
usa_interstates = interstates_gdf.union_all()

## Merge data

In [9]:
states_df["interstates"] = states_df.apply(
    lambda row: row["geometry"].intersection(usa_interstates), axis=1
)

In [10]:
states_df = states_df.dropna()
states_interstates_df = states_df[["NAME", "STUSPS", "geometry", "interstates"]]

In [50]:
states_interstates_population_df = states_with_population_df.merge(
    states_interstates_df, on="STUSPS", how="inner"
)
states_interstates_population_df["POPULATION"] = states_interstates_population_df[
    "POPULATION"
].astype(int)

In [51]:
def utm_crs_from_latlon(lat, lon):
    crs_params = dict(
        proj="utm", zone=utm.latlon_to_zone_number(lat, lon), south=lat < 0
    )
    return pyproj.crs.CRS.from_dict(crs_params)

In [52]:
def calculate_miles(row) -> int:
    wgs84 = pyproj.CRS("EPSG:4326")
    utm = pyproj.CRS(f'EPSG:{row["CRS"]}')
    project = pyproj.Transformer.from_crs(wgs84, utm, always_xy=True).transform
    transformed_interstates = transform(project, row["interstates"])
    return int(round(transformed_interstates.length * 0.000621371))


def calculate_area(row) -> int:
    wgs84 = pyproj.CRS("EPSG:4326")
    utm = pyproj.CRS(f'EPSG:{row["CRS"]}')
    project = pyproj.Transformer.from_crs(wgs84, utm, always_xy=True).transform
    transformed_state = transform(project, row["geometry"])
    return transformed_state.area

In [53]:
states_interstates_population_df["CRS"] = states_interstates_population_df.apply(
    lambda row: utm_crs_from_latlon(
        row["geometry"].centroid.y, row["geometry"].centroid.x
    ).to_epsg(),
    axis=1,
)

In [54]:
states_interstates_population_df[
    "interstate_length"
] = states_interstates_population_df.apply(lambda row: calculate_miles(row), axis=1)

In [55]:
states_interstates_population_df["state_area"] = states_interstates_population_df.apply(
    lambda row: calculate_area(row), axis=1
)
states_interstates_population_df["sqmi"] = (
    (states_interstates_population_df["state_area"] * 3.861e-7)
    .round(decimals=0)
    .astype(int)
)

In [56]:
states_interstates_population_df["miles_interstate_per_sqmi"] = (
    states_interstates_population_df["interstate_length"]
    / states_interstates_population_df["sqmi"]
)

In [57]:
states_interstates_population_df["per_1000"] = states_interstates_population_df[
    "interstate_length"
] / (states_interstates_population_df["POPULATION"] / 1000)
states_interstates_population_df["per_10k"] = states_interstates_population_df[
    "interstate_length"
] / (states_interstates_population_df["POPULATION"] / 10_000)
states_interstates_population_df["per_100k"] = states_interstates_population_df[
    "interstate_length"
] / (states_interstates_population_df["POPULATION"] / 100_000)
states_interstates_population_df["per_1m"] = states_interstates_population_df[
    "interstate_length"
] / (states_interstates_population_df["POPULATION"] / 1_000_000)

In [58]:
states_interstates_population_df = states_interstates_population_df[
    [
        "NAME",
        "geometry",
        "miles_interstate_per_sqmi",
        "interstate_length",
        "per_1000",
        "per_10k",
        "per_100k",
        "per_1m",
    ]
]

## Export

In [59]:
states_interstates_population_gdf = gpd.GeoDataFrame(
    states_interstates_population_df, crs=4326
)
states_interstates_population_gdf = states_interstates_population_gdf.to_crs(9311)

In [60]:
states_interstates_population_gdf.to_file("data/states_with_counts.gpkg")